# MLP Language Model
__Bengio et. al.__

* Longer contexts i.e. 3-gram, 4-gram etc. result in the word matrix `N` to blow-up in size and dimensions.
* Take a token-embedding instead - swapping out the sparse representation for a dense one.
* This embedding matrix `C` will be of dimension (vocabulary) X (dimension size)
    * `dimension size` is a hyperparameter.

In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
words = open("../names.txt").read().splitlines()
chars = sorted(list(set(''.join(words))))
stoi = dict([(c, i) for i, c in enumerate(['.']+chars)])
itos = dict([(i, c) for c, i in stoi.items()])
vocab_size = len(stoi.keys())

# Architecture
* 3-gram context
* Hyperparameters - embedding dim size, hidden layer dim

In [ ]:
block_size = 3
def build_dataset(words, block_size=3):
    # Constructing the input matrix
    X, Y = [],[]
    
    for w in words:
        context = [0] * block_size
        for ch in w + ".":
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            #print("".join([itos[i] for i in context]) + " ---> " + ch)
            context = context[1:] + [ix]
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xtest, Ytest = build_dataset(words[n2:])

In [ ]:
Xtr.shape, Ytr.shape, Xdev.shape, Ydev.shape, Xtest.shape, Ytest.shape

In [ ]:
# Embedding matrix
emb_dim = 10
hidden_dim = 200

g = torch.Generator().manual_seed(2147483647)
C = torch.randn(vocab_size, emb_dim, generator=g, requires_grad=True)
W1 = torch.randn((block_size * emb_dim, hidden_dim), generator=g, requires_grad=True)
b1 = torch.randn(hidden_dim, generator=g, requires_grad=True)
W2 = torch.randn((hidden_dim, vocab_size), generator=g, requires_grad=True)
b2 = torch.randn(vocab_size, generator=g, requires_grad=True)
parameters = [C, W1, b1, W2, b2]
# Decay lr array
#lre = torch.linspace(-3, 0, epochs)
#lrs = 10**lre

lri = []
lossi = []
stepi = []

minibatch_size = 32

In [ ]:
sum(p.nelement() for p in parameters)

In [ ]:
epochs = 40000
lr = 0.01
for epoch in range(epochs):
    ix = torch.randint(0, Xtr.shape[0], (32,))
    
    # Forward pass
    emb = C[Xtr[ix]]
    h = torch.tanh(emb.view(-1, block_size * emb_dim) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Ytr[ix])
    
    # Backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    
    # Update weights
    #lr = lrs[epoch]
    #lr = 0.1 if epoch < 100000 else 0.01
    for p in parameters:
        p.data += -lr * p.grad

    # track stats
    stepi.append(epoch)
    lossi.append(loss.log10().item())

print(loss.item())

In [ ]:
emb = C[Xtr]
h = torch.tanh(emb.view(-1, block_size * emb_dim) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ytr)
print(f"{loss.item()=}")

In [ ]:
emb = C[Xdev]
h = torch.tanh(emb.view(-1, block_size * emb_dim) @ W1 + b1)
logits = h @ W2 + b2
devloss = F.cross_entropy(logits, Ydev)
print(f"{devloss.item()=}")

In [ ]:
emb = C[Xtest]
h = torch.tanh(emb.view(-1, block_size * emb_dim) @ W1 + b1)
logits = h @ W2 + b2
devloss = F.cross_entropy(logits, Ytest)
print(f"{devloss.item()=}")

In [ ]:
plt.plot(stepi, lossi)

In [ ]:
plt.plot(lre, lossi)
# -1.0 is where it minimizes so this is a good learning rate.

In [ ]:
# visualize dimensions 0 and 1 of the embedding matrix C for all characters
plt.figure(figsize=(8,8))
plt.scatter(C[:,0].data, C[:,1].data, s=200)
for i in range(C.shape[0]):
    plt.text(C[i,0].item(), C[i,1].item(), itos[i], ha="center", va="center", color='white')
plt.grid('minor')

In [ ]:
	
# sample from the model
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):
    
    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      emb = C[torch.tensor([context])] # (1,block_size,d)
      h = torch.tanh(emb.view(1, -1) @ W1 + b1)
      logits = h @ W2 + b2
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break
    
    print(''.join(itos[i] for i in out))